In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('titanic_train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [5]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [6]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [8]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
30,1,male,40.0,0,0,27.7208,C
10,3,female,4.0,1,1,16.7000,S
873,3,male,47.0,0,0,9.0000,S
182,3,male,9.0,4,2,31.3875,S
876,3,male,20.0,0,0,9.8458,S


In [9]:
numerical_features = ['Age', 'Fare']
categorical_features = ['Embarked','Sex']

In [13]:
numerical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),('scaler', StandardScaler())
],verbose=True)

In [14]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
], verbose=True) 

In [19]:
preprocessor = ColumnTransformer(
    transformers=[
    ('num', numerical_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
],verbose=True)

In [20]:
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])


In [24]:
from sklearn import set_config

set_config(display='diagram')
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())],
                                                           verbose=True),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))],
                                                           verbose=True),
                                                  ['Embarked', 'Sex'])],
                                   verbose=True)),
                ('classifier', LogisticRegression())])

In [23]:
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())],
                                                           verbose=True),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))

In [26]:
param_grid = {
    'preprocessor__num__imputer__strategy':['mean', 'median'],
    'preprocessor__cat__imputer__strategy':['most_frequent', 'constant'],
    'classifier__C': [0.1, 1.0, 10, 100]
}

grid_search = GridSearchCV(clf, param_grid, cv = 20)

In [27]:
grid_search.fit(X_train, y_train)

print(f'Best Params:')
print(grid_search.best_params_)

[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing scaler, total=   0.0s
[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing ohe, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s
[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing scaler, total=   0.0s
[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing ohe, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s
[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing scaler, total=   0.0s
[Colum

In [29]:
print(f'Internal CV Score: {grid_search.best_score_: .3f}')

Internal CV Score:  0.788


In [33]:
import pandas as pd

cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results = cv_results.sort_values('mean_test_score', ascending=False)
cv_results[['param_classifier__C',
            'param_preprocessor__cat__imputer__strategy',
            'param_preprocessor__num__imputer__strategy',
            'mean_test_score']]

,param_classifier__C,param_preprocessor__cat__imputer__strategy,param_preprocessor__num__imputer__strategy,mean_test_score
4,1.0,most_frequent,mean,0.787897
5,1.0,most_frequent,median,0.787897
6,1.0,constant,mean,0.787897
7,1.0,constant,median,0.787897
8,10,most_frequent,mean,0.787897
9,10,most_frequent,median,0.787897
10,10,constant,mean,0.787897
11,10,constant,median,0.787897
12,100,most_frequent,mean,0.787897
13,100,most_frequent,median,0.787897


In [34]:
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,param_preprocessor__cat__imputer__strategy,param_preprocessor__num__imputer__strategy,params,split0_test_score,split1_test_score,...,split13_test_score,split14_test_score,split15_test_score,split16_test_score,split17_test_score,split18_test_score,split19_test_score,mean_test_score,std_test_score,rank_test_score
4,0.018641,0.005088,0.003983,0.004687,1.0,most_frequent,mean,"{'classifier__C': 1.0, 'preprocessor__cat__imp...",0.805556,0.722222,...,0.885714,0.771429,0.685714,0.8,0.742857,0.828571,0.742857,0.787897,0.057255,1
5,0.019279,0.006027,0.006690,0.006980,1.0,most_frequent,median,"{'classifier__C': 1.0, 'preprocessor__cat__imp...",0.805556,0.722222,...,0.885714,0.771429,0.685714,0.8,0.742857,0.828571,0.742857,0.787897,0.057255,1
6,0.020599,0.007124,0.005023,0.006174,1.0,constant,mean,"{'classifier__C': 1.0, 'preprocessor__cat__imp...",0.805556,0.722222,...,0.885714,0.771429,0.685714,0.8,0.742857,0.828571,0.742857,0.787897,0.057255,1
7,0.018912,0.006837,0.007090,0.006953,1.0,constant,median,"{'classifier__C': 1.0, 'preprocessor__cat__imp...",0.805556,0.722222,...,0.885714,0.771429,0.685714,0.8,0.742857,0.828571,0.742857,0.787897,0.057255,1
8,0.018730,0.005321,0.004805,0.006318,10,most_frequent,mean,"{'classifier__C': 10, 'preprocessor__cat__impu...",0.805556,0.722222,...,0.885714,0.771429,0.685714,0.8,0.742857,0.828571,0.742857,0.787897,0.057255,1
9,0.018207,0.004868,0.007426,0.006754,10,most_frequent,median,"{'classifier__C': 10, 'preprocessor__cat__impu...",0.805556,0.722222,...,0.885714,0.771429,0.685714,0.8,0.742857,0.828571,0.742857,0.787897,0.057255,1
10,0.019775,0.005863,0.006295,0.006386,10,constant,mean,"{'classifier__C': 10, 'preprocessor__cat__impu...",0.805556,0.722222,...,0.885714,0.771429,0.685714,0.8,0.742857,0.828571,0.742857,0.787897,0.057255,1
11,0.026412,0.008035,0.006802,0.004670,10,constant,median,"{'classifier__C': 10, 'preprocessor__cat__impu...",0.805556,0.722222,...,0.885714,0.771429,0.685714,0.8,0.742857,0.828571,0.742857,0.787897,0.057255,1
12,0.019563,0.003888,0.007044,0.003630,100,most_frequent,mean,"{'classifier__C': 100, 'preprocessor__cat__imp...",0.805556,0.722222,...,0.885714,0.771429,0.685714,0.8,0.742857,0.828571,0.742857,0.787897,0.057255,1
13,0.019678,0.005303,0.007494,0.002417,100,most_frequent,median,"{'classifier__C': 100, 'preprocessor__cat__imp...",0.805556,0.722222,...,0.885714,0.771429,0.685714,0.8,0.742857,0.828571,0.742857,0.787897,0.057255,1
